In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core:PipelineRun._from_dto with exception (azureml-core 1.0.83 (/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.57.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core:StepRun._from_reused_dto with exception (azureml-core 1.0.83 (/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.57.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core:StepRun._from_dto with exception (azureml-core 1.0.83 (/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.57.*')).


SDK version: 1.0.83


In [2]:
import os
import sys

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
    
from utils_nlp.azureml import azureml_utils

from azureml.core import Experiment, Workspace, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails
from azureml.train.dnn import Nccl

In [3]:
# Create the workspace using the specified parameters
ws = Workspace.create(name = "daden1amlws",
                      subscription_id = "9086b59a-02d7-4687-b3fd-e39fa5e0fd9b",
                      resource_group = "daden1aml", 
                      location = "eastus2",
                      create_resource_group = False,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

In [4]:
# Retrieve the workspace
ws = Workspace.setup()

# Print the workspace attributes
print('Workspace name: ' + ws.name, 
      'Workspace region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

setup() is now deprecated. Instead, please use create() to create a new workspace, or get()/from_config() to retrieve an existing one


Workspace name: daden1amlws
Workspace region: eastus2
Subscription id: 9086b59a-02d7-4687-b3fd-e39fa5e0fd9b
Resource group: daden1aml


In [5]:
AMLCOMPUTE_CLUSTER_NAME = "extsum5"
NODE_COUNT = 4
VM_SIZE = 'STANDARD_NC6'

AZUREML_CONFIG_PATH = "./.azureml"
EXPERIMENT_NAME = 'NLP-ExtSum'
PROJECT_FOLDER = './azureml_exp'

TARGET_DATA_FOLDER = '/temp_data5'
SOURCE_DATA_FOLDER = '/dadendev/temp_data5/'
OUTPUT_DIR = './output'

os.makedirs(PROJECT_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, EXPERIMENT_NAME), exist_ok=True)

In [6]:

#!cp sum_dis_train_aml.py './azureml_exp'
#!cp -r ../../utils_nlp './azureml_exp'
#!cp ../../tools/nlp_gpu.yaml './azureml_exp'

In [7]:
ds = ws.get_default_datastore()

In [42]:
ds.upload(src_dir=SOURCE_DATA_FOLDER, target_path=TARGET_DATA_FOLDER)

I0124 05:32:35.001156 140175900309312 azure_storage_datastore.py:694] Called AzureBlobDatastore.upload
I0124 05:32:36.542125 140175900309312 azure_storage_datastore.py:323] Uploading an estimated of 23144 files
I0124 05:32:36.739460 140156023092992 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.22.bert.pt


Uploading an estimated of 23144 files
Uploading /dadendev/temp_data5/cnndm.train.22.bert.pt


I0124 05:32:36.773340 140156728080128 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.78.bert.pt
I0124 05:32:36.806787 140169512466176 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.88.bert.pt
I0124 05:32:36.840937 140158263355136 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.test.1.bert.pt
I0124 05:32:36.875894 140165762762496 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.51.bert.pt
I0124 05:32:36.909395 140156493084416 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.5.bert.pt
I0124 05:32:36.941838 140156258088704 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.120.bert.pt


Uploading /dadendev/temp_data5/cnndm.train.78.bert.pt
Uploading /dadendev/temp_data5/cnndm.train.88.bert.pt
Uploading /dadendev/temp_data5/cnndm.test.1.bert.pt
Uploading /dadendev/temp_data5/cnndm.train.51.bert.pt
Uploading /dadendev/temp_data5/cnndm.train.5.bert.pt
Uploading /dadendev/temp_data5/cnndm.train.120.bert.pt


I0124 05:32:36.975453 140155788097280 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.114.bert.pt
I0124 05:32:37.011805 140155779704576 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.140.bert.pt
I0124 05:32:37.045219 140155771311872 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.87.bert.pt
I0124 05:32:37.080537 140155762919168 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.104.bert.pt
I0124 05:32:37.114509 140155754526464 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.94.bert.pt
I0124 05:32:37.146740 140155746133760 azure_storage_datastore.py:323] Uploading /dadendev/temp_data5/cnndm.train.59.bert.pt


...

W0124 05:36:21.654666 140155662206720 connectionpool.py:748] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')",))': /azureml-blobstore-b4da5ba6-4dd0-43d9-b818-c99833e3ad7e/cnndm/tmpqulb9mtv/system/cand.10286.txt
I0124 05:36:21.825634 140155662206720 azure_storage_datastore.py:323] Uploaded /dadendev/temp_data5/tmpqulb9mtv/system/cand.10286.txt, 4706 files out of an estimated total of 23144


Uploaded /dadendev/temp_data5/tmpqulb9mtv/system/cand.10286.txt, 4706 files out of an estimated total of 23144


W0124 05:36:32.700609 140155611850496 connectionpool.py:748] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')",))': /azureml-blobstore-b4da5ba6-4dd0-43d9-b818-c99833e3ad7e/cnndm/tmpqulb9mtv/system/cand.9596.txt
I0124 05:36:32.864541 140155611850496 azure_storage_datastore.py:323] Uploaded /dadendev/temp_data5/tmpqulb9mtv/system/cand.9596.txt, 4707 files out of an estimated total of 23144


Uploaded /dadendev/temp_data5/tmpqulb9mtv/system/cand.9596.txt, 4707 files out of an estimated total of 23144


KeyboardInterrupt: 

In [8]:
try:
    gpu_compute_target = ComputeTarget(workspace=ws, name=AMLCOMPUTE_CLUSTER_NAME)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE,
                                                           max_nodes=NODE_COUNT,
                                                           NodeIdleTimeBeforeScaleDown='PT1200S')

    # create the cluster
    gpu_compute_target = ComputeTarget.create(ws, AMLCOMPUTE_CLUSTER_NAME, compute_config)

    gpu_compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(gpu_compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-01-24T06:23:13.803000+00:00', 'errors': None, 'creationTime': '2020-01-23T04:50:26.160743+00:00', 'modifiedTime': '2020-01-23T20:31:35.349184+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1200S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [9]:
experiment = Experiment(ws, name=EXPERIMENT_NAME)

In [18]:
NcclConfig=Nccl()
ENTRY_SCRIPT = "extractive_summarization_cnndm_distributed_train.py"
estimator = PyTorch(source_directory=PROJECT_FOLDER,
                    compute_target=gpu_compute_target,
                    script_params={
                        "--dist_url": "$AZ_BATCHAI_PYTORCH_INIT_METHOD",
#                         "--dist_url": "$AZ_BATCH_MASTER_NODE",
                        "--rank": "$AZ_BATCHAI_TASK_INDEX",
                        "--node_count": 4,
                        "--data_dir":ds.path(f'{TARGET_DATA_FOLDER}').as_mount(),
                        "--cache_dir": ds.path(f'cache/{EXPERIMENT_NAME}/').as_mount(),
                        '--output_dir':ds.path(f'output/{EXPERIMENT_NAME}/').as_mount(),
                        #"--quick_run": True
                        "--quick_run": False
                    },
                    entry_script= ENTRY_SCRIPT,
                    node_count=NODE_COUNT,
                    distributed_training=NcclConfig,
                    conda_dependencies_file="nlp_gpu.yaml",
                    use_gpu=True)

If environment_definition or conda_dependencies_file is specified, Azure ML will not install any framework related packages on behalf of the user.
framework_version is not specified, defaulting to version 1.3.


In [19]:
run = experiment.submit(estimator)

Submitting /dadendev/nlp/examples/text_summarization/azureml_exp directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


In [20]:

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [10]:
# Downloading log file to run the perf benchmarking script
fetched_run = Run(experiment, "NLP-ExtSum_1579816237_ea238f69")

In [14]:
ds.download(target_path=OUTPUT_DIR,
                   prefix=f'temp_data5/generated_summaries.txt',
                   show_progress=True)

Downloaded temp_data5/generated_summaries.txt, 1 files out of an estimated total of 1


1

In [11]:
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
I0124 04:05:49.220191 140175900309312 file_utils.py:39] PyTorch version 1.3.0 available.


In [12]:
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import ExtSumProcessedData

In [13]:
#CNNDMBertSumProcessedData.download(local_path="/dadendev/temp_data5")
train_dataset, test_dataset = ExtSumProcessedData().splits(root="/dadendev/temp_data5")

In [14]:
target = [i['tgt_txt'] for i in test_dataset]

In [29]:
len(test_dataset)

11489

In [16]:
prediction = []
with open(os.path.join(OUTPUT_DIR, "temp_data5/generated_summaries.txt"), "r") as filehandle:
    for cnt, line in enumerate(filehandle):
        #print(line)
        prediction.append(line[0:-1]) # remove the ending "\n"
        #break

In [17]:
assert len(prediction)==len(target)

In [25]:
prediction[0]

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .<q>he was flown back to chicago via air ambulance on march 20 , but he died on sunday .<q>a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .'

In [15]:
target[0]

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'

In [18]:
RESULT_DIR = './testrouge'

In [19]:
rouge_score = get_rouge(prediction, target, RESULT_DIR)

11489
11489


2020-01-24 04:06:04,512 [MainThread  ] [INFO ]  Writing summaries.
I0124 04:06:04.512003 140175900309312 pyrouge.py:525] Writing summaries.
2020-01-24 04:06:04,522 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ./testrouge/tmpjnl9bobx/system and model files to ./testrouge/tmpjnl9bobx/model.
I0124 04:06:04.522701 140175900309312 pyrouge.py:518] Processing summaries. Saving system files to ./testrouge/tmpjnl9bobx/system and model files to ./testrouge/tmpjnl9bobx/model.
2020-01-24 04:06:04,523 [MainThread  ] [INFO ]  Processing files in ./testrouge/rouge-tmp-2020-01-24-04-06-03/candidate/.
I0124 04:06:04.523659 140175900309312 pyrouge.py:43] Processing files in ./testrouge/rouge-tmp-2020-01-24-04-06-03/candidate/.
2020-01-24 04:06:05,646 [MainThread  ] [INFO ]  Saved processed files to ./testrouge/tmpjnl9bobx/system.
I0124 04:06:05.646162 140175900309312 pyrouge.py:53] Saved processed files to ./testrouge/tmpjnl9bobx/system.
2020-01-24 04:06:05,647 [MainThread  ] [IN

---------------------------------------------
1 ROUGE-1 Average_R: 0.52854 (95%-conf.int. 0.52570 - 0.53134)
1 ROUGE-1 Average_P: 0.35629 (95%-conf.int. 0.35395 - 0.35848)
1 ROUGE-1 Average_F: 0.41146 (95%-conf.int. 0.40933 - 0.41352)
---------------------------------------------
1 ROUGE-2 Average_R: 0.23487 (95%-conf.int. 0.23217 - 0.23771)
1 ROUGE-2 Average_P: 0.15804 (95%-conf.int. 0.15608 - 0.16005)
1 ROUGE-2 Average_F: 0.18238 (95%-conf.int. 0.18024 - 0.18455)
---------------------------------------------
1 ROUGE-L Average_R: 0.48060 (95%-conf.int. 0.47787 - 0.48335)
1 ROUGE-L Average_P: 0.32453 (95%-conf.int. 0.32228 - 0.32658)
1 ROUGE-L Average_F: 0.37451 (95%-conf.int. 0.37243 - 0.37657)

